In [1]:
%load_ext autoreload
%autoreload 2

import os 
os.environ["LOGURU_LEVEL"] = "INFO"

from oxeo.core.models.data import data2gdf, fetch_water_list
from oxeo.core.models.tile import get_all_paths
from oxeo.core.models.waterbody import get_waterbodies
from oxeo.water.testsuite.testsuite import PixelTestWaterBody, TrainingPixelTestWaterBody
from oxeo.water.models.waterbody_predictor import WaterBodyPredictor

In [2]:
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')


In [3]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/geom')
data = fetch_water_list([25906117, -2939785], engine)
gdf = data2gdf(data)

In [4]:

constellations=["sentinel-2", "landsat-8"]
root_dir="/home/fran/tiles"
all_paths = get_all_paths(gdf, constellations, root_dir)

/home/fran/miniconda3/envs/oxeo/lib/python3.9/site-packages/sentinelhub/areas.py:105: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  return shapely.ops.cascaded_union(shape_list)
2022-03-02 13:58:45,093 |     INFO | tile:get_all_paths:128 - All tiles for the supplied geometry: ['/home/fran/tiles/34_H_10000_26_623/sentinel-2', '/home/fran/tiles/34_H_10000_26_623/landsat-8']


In [5]:
waterbodies = get_waterbodies(gdf, constellations, root_dir)

/home/fran/miniconda3/envs/oxeo/lib/python3.9/site-packages/sentinelhub/areas.py:105: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  return shapely.ops.cascaded_union(shape_list)


In [6]:
waterbodies[1].paths[0].mask_path

'/home/fran/tiles/34_H_10000_26_623/sentinel-2/mask'

In [7]:
# This Test just load waterbody masks from bucket or local. 
# It doesn't perform the prediction.
ptwb = PixelTestWaterBody(y_true_mask_name = "cnn",
                          y_pred_mask_name = "cnn",
                          waterbody=waterbodies[1], 
                          start_date='2019-01-01', 
                          end_date='2019-01-30', 
                          metrics= {"metric_1": lambda x,y: [1]*len(x),
                                    "metric_2": lambda x,y: [2]*len(x)},
                          fs=None)

In [ ]:
ptwb.calculate_metrics()

In [8]:
# This Test load a waterbody mask for the y_true from bucket or local. 
# It performs a prediction using a given predictor

import gcsfs
fs = gcsfs.GCSFileSystem()
wbp = WaterBodyPredictor(fs, "cnn", revisit_chunk_size=5, 
                         ckpt_path="gs://oxeo-models/semseg/epoch_012.ckpt",
                         batch_size=4,
                         bands=["nir", "red", "green", "blue", "swir1", "swir2"],
                         target_size=1000
                         )

ttwb = TrainingPixelTestWaterBody(predictor=wbp, y_true_mask_name = "cnn",
                          waterbody=waterbodies[1], 
                          start_date='2019-01-01', 
                          end_date='2019-01-30', 
                          metrics= {"metric_1": lambda x,y: [1]*len(x),
                                    "metric_2": lambda x,y: [2]*len(x)},
                          fs=None)

ttwb.calculate_metrics()

2022-03-02 13:59:00,467 |     INFO | timeseries:merge_masks_all_constellations:35 - merge_masks_all_constellations; constellation='sentinel-2'; waterbody.area_id=25906117: merging
2022-03-02 13:59:00,468 |     INFO | timeseries:merge_masks_one_constellation:59 - merge_masks_one_constellation; constellation='sentinel-2'; 25906117: get details
2022-03-02 13:59:00,469 |     INFO | tile:get_patch_size:84 - Loading to check size arr_path='/home/fran/tiles/34_H_10000_26_623/sentinel-2/data'
2022-03-02 13:59:00,472 |     INFO | timeseries:merge_masks_one_constellation:71 - paths ['/home/fran/tiles/34_H_10000_26_623']
2022-03-02 13:59:00,473 |     INFO | timeseries:merge_masks_one_constellation:74 - merge_masks_one_constellation; constellation='sentinel-2'; 25906117: create dataarray
2022-03-02 13:59:00,520 |     INFO | timeseries:merge_masks_all_constellations:35 - merge_masks_all_constellations; constellation='landsat-8'; waterbody.area_id=25906117: merging
2022-03-02 13:59:00,521 |     INFO

,metric_1,metric_2,constellation,timestamp
0,1,2,sentinel-2,2019-01-04 08:59:17.000000
1,1,2,sentinel-2,2019-01-05 08:59:17.000000
2,1,2,sentinel-2,2019-01-09 08:59:17.000000
3,1,2,sentinel-2,2019-01-10 08:59:17.000000
4,1,2,sentinel-2,2019-01-14 08:59:17.000000
5,1,2,sentinel-2,2019-01-15 08:59:17.000000
6,1,2,sentinel-2,2019-01-19 08:59:17.000000
7,1,2,sentinel-2,2019-01-20 08:59:17.000000
8,1,2,sentinel-2,2019-01-24 08:59:17.000000
9,1,2,sentinel-2,2019-01-25 08:59:17.000000
